In [45]:
from langchain.chat_models import ChatOpenAI
'''
.env에 있는 'OPENAI_API_KEY' variable을 자동으로 불러와 모델 객체를 생성할 때 사용한다.
만약 .env 파일명을 다르게 작성하거나 'OPENAI_API_KEY' 가 아닌 다른 variable name을 사용했다면 오류 발생.
위의 방법 말고도 "OpenAI(openai_api_key = "{발급한 API_KEY}")"와 같이 사용도 가능하다.
'''

chat = ChatOpenAI(temperature=0.1, 
                #   model= "gpt-4o",
                  )

In [6]:
'''predict_messages 메서드를 사용하여 message들로 이루어진 List를 전달하는 방식'''
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# SystemMessage : 우리가 LLM에 설정들을 제공하기 위한 Message(e.g, 역할 부여, 작업 전제 등)
# HumanMessage : 사용자가 AI에게 보내는 Message
# AIMessage : AI에 의해 보내지는 Message

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian",),
    AIMessage(content="Ciao, mi chiamo Paolo!",),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?",),
]

chat.predict_messages(messages)

AIMessage(content='Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Mi chiamo Paolo, sono un esperto di geografia. Posso aiutarti con altre domande?')

In [11]:
'''
PromptTemplate은 String 으로
ChatPromptTemplate은 template message 로부터 만든다.'''
from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?",)

prompt = template.format(country_a = "Mexico", country_b = "Thailand")

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line.'

In [14]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}"),
        ("ai", "Ciao, mi chiamo {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
    ]
)

prompt = template.format_messages(
    language= "English", 
    name= "Hanni", 
    country_a="한국", 
    country_b="호주",
    )

chat.predict_messages(prompt)

AIMessage(content='Hello! The distance between South Korea (한국) and Australia (호주) is approximately 7,800 kilometers. My name is Hanni. How can I assist you today?')

In [15]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(slef, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [28]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", 
         "You are a list generating machine. \
             Everything you are asked will be answered with a comma separated list of max {max_items}. Do Not reply with anything else"),
        ("human", 
         "{question}"),
    ]
)

prompt = template.format_messages(
    max_items = 10,
    question = "What are the colors?",
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)


['Red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [46]:
'''
- chain을 활용해 위에서 작성한 코드를 간결하게 구현 가능하다.
(format_messages, predict_message, CommaOutputParser 인스턴스 정의 및 parse 메서드 실행)
- 입력하는 element 들을 순서대로 실행한다. 
- chain 간의 결합도 가능하다.
'''

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items" : 6,
    "question" : "Who are the member of NewJeans?"
})

['Sarah', 'Michael', 'Emily', 'David', 'Jessica', 'Chris']

### 복수의 Chain 연결

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, 
    streaming=True, # LLM의 응답(response)이 생성되는대로 볼 수 있게 해준다.
    callbacks=[StreamingStdOutCallbackHandler()] # 볼 수 있는 문자가 생길 때 마다 print
    )

chef_template = ChatPromptTemplate.from_messages([
    ("system", "You are world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find"),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_template | chat

veg_chef_template = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_template | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "Indian"})


Great choice! Here is a simple recipe for Chicken Tikka Masala, a popular Indian dish:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons paprika
- 1 teaspoon ground cinnamon
- 1 teaspoon ground turmeric
- 1 teaspoon ground coriander
- 1 teaspoon cayenne pepper (adjust to taste)
- Salt and pepper to taste
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon grated ginger
- 1 can (14 oz) crushed tomatoes
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, cumin, paprika, cinnamon, turmeric, coriander, cayenne pepper, salt, and pepper. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or overnight for best results.

2. Preheat the oven to 400°F (200°C). Place the marinated chicken pieces on a baki

AIMessageChunk(content="As a vegetarian chef, I can offer you a delicious vegetarian alternative to Chicken Tikka Masala using paneer instead of chicken. Paneer is a fresh cheese common in Indian cuisine and is a great substitute for meat in many dishes. Here's how you can modify the recipe:\n\nIngredients:\n- 1 lb paneer, cut into bite-sized pieces\n- 1 cup plain yogurt\n- 2 tablespoons lemon juice\n- 2 teaspoons ground cumin\n- 2 teaspoons paprika\n- 1 teaspoon ground cinnamon\n- 1 teaspoon ground turmeric\n- 1 teaspoon ground coriander\n- 1 teaspoon cayenne pepper (adjust to taste)\n- Salt and pepper to taste\n- 2 tablespoons vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1 tablespoon grated ginger\n- 1 can (14 oz) crushed tomatoes\n- 1 cup heavy cream\n- Fresh cilantro, chopped (for garnish)\n\nInstructions:\n1. Follow the same marinating process as the original recipe, substituting the chicken with paneer. Marinate the paneer in the yogurt and spice mixture